# 本章简介
另一种常用于降维方法就是特征抽取，特征抽取可以将原始数据集变化到一个维度更低的新的特征子空间，从而对数据进行压缩，避免维度灾难  
本章读者将学习三种降维技术：  
+ 无监督数据压缩——主成分分析(PCA)
+ 基于类别可分最大化的监督降维技术——线性判别分析(LDA)
+ 通过核主成分分析进行非线性降维 5  

# 5.1 无监督数据降维技术-主成分分析
> 本章介绍了主成分分析(PCA)的原理、实现和使用方法

### 问题：特征选择和特征抽取有什么区别？
> 特征抽取后的新特征是原来特征的一个映射，而特征选择后的特征是原来特征的一个子集 5  

特征抽取算法会将数据转换或者映射到一个新的特征空间，可以理解为：在尽可能保持相关信息的情况下，对数据进行压缩的一种方法  
特征抽取通常用于提高计算效率，同样也可以帮助我们降低"维度灾难"——尤其当模型不适于正则化处理时  
主成分分析(PCA)是一种广泛应用于不同领域的无监督线性数据转换技术，其突出作用是降维  
简而言之，PCA的目标是在高维数据中找到最大方差的方向，并将数据映射到一个维度不大于原始数据的新的子空间上 5  
如下图所示，新的子空间上正交的坐标轴（主成分）可被解释为方差最大的方向。在此，$x_1$和$x_2$为原始特征的坐标轴，而PC1和PC2即为主成分  
![5-1](../syn_pic/py_machine_learning/5-1.png)
如果使用PCA降维，我们将构建一个$d\times k$维的转换矩阵W，这样就可以将一个样本向量x映射到一个新的k维特征子空间上去，此空间的维度小于原始的d维特征空间：  
$$x=[x_1,x_2,\dots,x_d]，x\in{R}^d$$
$$\downarrow xW,W\in{R}^{d\times k}$$
$$z=[z_1,z_2,\dots,z_k],z\in{R}^k$$
完成从原始d维数据到新的k维子空间(一般情况下$k\ll d$)的转换后，第一主成分的方差应该是最大的，由于各主成分之间是不相关的（正交的），后续各主成分也具备尽可能大的方差  
需要注意的是，主成分的方向对数据值的范围高度敏感，如果特征的值使用不同的度量标准，我们需要先对特征进行标准化处理  
我们先通过以下几个步骤来概括以下算法的流程：
1. 对原始d维数据集做标准化处理 5\*2    
2. 构造样本的协方差矩阵
3. 计算协方差矩阵的特征值和相应的特征向量
4. 选择与前k个最大特征值对应的特征向量，其中k为新特征空间的维度($k\le d$)
5. 通过前k个特征向量构建映射矩阵W
6. 通过映射矩阵W将d维的输入数据集X转换到新的k维特征子空间 5  

### 问题：如何理解主成分分析降维的原理？
> 主成分简单来说就是对结果影响最大的成分，而成分包含在各个特征中  
特征抽取的含义就是把成分从特征中抽取出来，将成分重新组合后映射到新的k维特征空间中  
这种抽取并组合的原理是通过计算贡献方差来找出主成分，并通过映射矩阵将数据转换到k维特征空间  
其中贡献方差是基于协方差矩阵转换成特征对中的特征值计算出来的，而映射矩阵是由特征对中的特征向量构造的 5  
  

## 5.1.1 总体方差与贡献方差
> 本节介绍了如何通过协方差矩阵转换成特征对（特征值和特征向量），并使用特征值计算贡献方差的方法

本节，我们将学习主成分分析算法的前四个步骤：数据标准化、构造协方差矩阵、获得协方差矩阵的特征值和特征向量，以及按降序排列特征值所对应的特征向量  

In [ ]:
import pandas as pd
'''
python	pandas	dataframe	to_csv() index

df_winecsv = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      header=None)
df_winecsv.columns = ['Class label', 'Alcohol', 
                   'Malic acid', 'Ash',
                  'Alcalinity of ash', 'Magnesium',
                  'Total phenols', 'Flavanoids',
                  'Nonflavanoid phenols',
                  'Proanthocyanins',
                  'Color intensity', 'Hue',
                  'OD280/OD315 of diluted wines',
                  'Proline']
df_winecsv.to_csv('../syc_data/py_machine_learning/wine.data', index=False)
df_winecsv.head()
'''

df_wine = pd.read_csv('../syc_data/py_machine_learning/wine.data')
df_wine.head()

我们将葡萄酒数据集划分为训练集和测试集，并使用单位方差将其标准化  

In [ ]:
'''
5
python	pandas	dataframe	values
python	sklearn	Model Selection	train_test_split()
python	sklearn	Preprocessing and Normalization	StandardScaler()
'''
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=0)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

完成数据预处理后，我们进入第二步：构造协方差矩阵  
$d\times d$维协方差矩阵，其中d位数据集的维度，此矩阵成对地存储了不同特征之间地协方差，例如两个特征$x_j和x_k$ 可通过如下公式来计算协方差：   
$$\sigma_{jk}=\frac{1}{n}\sum_{i=1}^n{(x_j^{(i)}-\mu_j)(x_k^{(i)}-\mu_k)}$$
在此，$\mu_j$和$\mu_k$分别位特征j和k的均值。注意，我们做了标准化处理后，样本的均值将为零   
两个特征之间的协方差如果位正，说明它们会同时增减，而一个负的协方差值则表示两个特征会朝相反的方向变动，一个包含三个特征的协方差矩阵可记为：5  
$$\Sigma=\left[\begin{matrix}\sigma^2_1&\sigma_{12}&\sigma_{13} \cr 
    \sigma_{21}&\sigma^2_2&\sigma_{23} \cr 
    \sigma_{31}&\sigma_{32}&\sigma^2_3 \end{matrix}\right]$$
协方差矩阵的特征向量代表主成分（最大方差方向），而对应的特征值大小就决定了特征向量的重要性  
### 问题：协方差矩阵其特征向量、特征值分别是什么？
> 特征向量和特征值是协方差矩阵通过特征分解后得到，如葡萄酒中13个特征变量——13x13协方差矩阵——13个维特征向量及13个特征值，特征向量以列的方式存储于一个13x13维的矩阵中 5  

就葡萄酒数据集来说，我们可以得到$13\times13$维协方差矩阵的13个特征向量及其对应的特征值。特征值V需满足如下条件：  
$$\Sigma{V}=\lambda{V}$$
5  
此处的特征值是$\lambda$一个标量，我们来计算协方差矩阵的特征对  
### 问题：什么是标量？
> 几何代数的概念，对应于矢量，标量亦称作无向量，用通俗的说法，标量是只有大小，没有方向的量 5  

In [ ]:
'''
python	numpy	The N-dimensional array (ndarray)	ndarray.T
python	numpy	Statistics	cov()
python	numpy	Linear algebra (numpy.linalg)	eig()
'''
import numpy as np
cov_mat = np.cov(X_train_std.T) # 转换成协方差矩阵
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat) # 将协方差矩阵进行特征分解得到特征值和特征向量
print('如下一共{:d}个特征值'.format(len(eigen_vals))) # eigen_vals特征值 eigen_vecs 特征向量
print('\nEigenvalues|特征值 \n{}'.format(eigen_vals))

### 问题：如何理解np.cov?
> 公式 $$cov(X,Y)=\frac{\Sigma^n_{i=1}{(X_i-\bar{X})(Y_i-\bar{Y})}}{n-1}$$
结果  $$C=\left[\begin{matrix}cov(1,1)&cov(1,2)&cov(1,3)&\dots&cov(1,n) \cr 
    cov(2,1)&cov(2,2)&cov(2,3)&\dots&cov(2,n) \cr 
    cov(3,1)&cov(3,2)&cov(3,3)&\dots&cov(3,n) \cr
    \dots&\dots&\dots&\dots&\dots \cr
    cov(n,1)&cov(n,2)&cov(n,3)&\dots&cov(n,n)
    \end{matrix}\right]$$

协方差矩阵计算的是不同维度之间的协方差，而不是不同样本之间。拿到一个样本矩阵，首先要明确的就是行代表什么，列代表什么，如下所示 5

In [ ]:
'''
5
python	numpy	Array manipulation routines	vstack()
'''
# 计算协方差的时候，一行代表一个特征
# 下面计算cov(T, S, M)
T = np.array([9, 15, 25, 14, 10, 18, 0, 16, 5, 19, 16, 20])
S = np.array([39, 56, 93, 61, 50, 75, 32, 85, 42, 70, 66, 80])
M = np.array([38, 56, 90, 63, 56, 77, 30, 80, 41, 79, 64, 88])
X = np.vstack((T, S, M))
# X每行代表一个属性
#  每列代表一个示例，或者说观测
print(np.cov(X))

因为要将数据集压缩到一个新的特征子空间上来实现数据降维，所以我们只选择那些包含最多信息（方差最大）的特征向量(主成分)组成子集  
由于特征值的大小决定了特征向量的重要性，因此需要将特征值按降序排列，我们感兴趣的是排序在前k个特征值所对应的特征向量。我们先绘制特征值的方差贡献率  
特征值$\lambda_j$的方差贡献率是指，特征值$\lambda_j$与所有特征值和的比值 5  
$$\frac{\lambda_j}{\sum_{j=1}^d{\lambda_j}}$$
我们可以计算并绘制出方差贡献率的累积分布图

In [ ]:
'''
python	Built-in Functions	sum()	sum()
python	Built-in Functions	sorted()	sorted() reverse
python	numpy	Mathematical functions	cumsum()
python	matplotlib	Pyplot function overview	bar() align
python	matplotlib	Pyplot function overview	step()
python	matplotlib	Pyplot function overview	legend() loc
m
'''
tot = sum(eigen_vals) # 计算所有特征值之和
var_exp = [(i/tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
import matplotlib.pyplot as plt
plt.bar(range(1,14), var_exp, alpha=0.5, align='center',
       label='individual explained variance')
plt.step(range(1,14), cum_var_exp, where='mid',
        label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show()

我们应注意：PCA是一种无监督方法，意味着我们可以忽略类标信息，方差度量的是特征值在轴线上的分布 5  
## 5.1.2 特征转换
> 本节介绍了如何通过特征向量构建映射矩阵

将协方差矩阵分解为特征对（特征值和特征向量）后，我们继续执行PCA方法的最后三个步骤，将葡萄酒数据集中的信息转换到新的主成分轴上  
我们将对特征值按降序进行排列，并通过挑选出对应的特征向量构造出映射矩阵，然后使用映射矩阵将数据转换到低维的子空间上  

In [ ]:
'''
python	numpy	Mathematical functions	abs()
python	Sequence Types — list, tuple, range	sort()	sort() reverse
'''
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
               for i in range(len(eigen_vals))]
eigen_pairs.sort(reverse=True)